In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [2]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Activation, BatchNormalization, SeparableConv2D, MaxPooling2D, add
from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler

import imgaug as ia
from imgaug import augmenters as iaa


Using TensorFlow backend.


In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
df.groupby('gender').count()

,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
gender,,,,,,,,
female,5937,5937,5937,5937,5937,5937,5937,5937
male,7636,7636,7636,7636,7636,7636,7636,7636


In [0]:
df.groupby('imagequality').count()

,gender,age,weight,carryingbag,footwear,emotion,bodypose,image_path
imagequality,,,,,,,,
Average,7509,7509,7509,7509,7509,7509,7509,7509
Bad,2240,2240,2240,2240,2240,2240,2240,2240
Good,3824,3824,3824,3824,3824,3824,3824,3824


In [0]:
df.groupby('age').count()

,gender,imagequality,weight,carryingbag,footwear,emotion,bodypose,image_path
age,,,,,,,,
15-25,2494,2494,2494,2494,2494,2494,2494,2494
25-35,5411,5411,5411,5411,5411,5411,5411,5411
35-45,3435,3435,3435,3435,3435,3435,3435,3435
45-55,1490,1490,1490,1490,1490,1490,1490,1490
55+,743,743,743,743,743,743,743,743


In [0]:
df.groupby('weight').count()

,gender,imagequality,age,carryingbag,footwear,emotion,bodypose,image_path
weight,,,,,,,,
normal-healthy,8628,8628,8628,8628,8628,8628,8628,8628
over-weight,891,891,891,891,891,891,891,891
slightly-overweight,3196,3196,3196,3196,3196,3196,3196,3196
underweight,858,858,858,858,858,858,858,858


In [0]:
df.groupby('carryingbag').count()

,gender,imagequality,age,weight,footwear,emotion,bodypose,image_path
carryingbag,,,,,,,,
Daily/Office/Work Bag,4603,4603,4603,4603,4603,4603,4603,4603
Grocery/Home/Plastic Bag,1321,1321,1321,1321,1321,1321,1321,1321
None,7649,7649,7649,7649,7649,7649,7649,7649


In [0]:
df.groupby('footwear').count()

,gender,imagequality,age,weight,carryingbag,emotion,bodypose,image_path
footwear,,,,,,,,
CantSee,5028,5028,5028,5028,5028,5028,5028,5028
Fancy,2507,2507,2507,2507,2507,2507,2507,2507
Normal,6038,6038,6038,6038,6038,6038,6038,6038


In [0]:
df.groupby('emotion').count()

,gender,imagequality,age,weight,carryingbag,footwear,bodypose,image_path
emotion,,,,,,,,
Angry/Serious,1500,1500,1500,1500,1500,1500,1500,1500
Happy,1609,1609,1609,1609,1609,1609,1609,1609
Neutral,9660,9660,9660,9660,9660,9660,9660,9660
Sad,804,804,804,804,804,804,804,804


In [0]:
df.groupby('bodypose').count()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,image_path
bodypose,,,,,,,,
Back,2207,2207,2207,2207,2207,2207,2207,2207
Front-Frontish,8383,8383,8383,8383,8383,8383,8383,8383
Side,2983,2983,2983,2983,2983,2983,2983,2983


In [0]:
count = df[df.gender == 'female']

In [0]:
count.shape[0]

5937

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.ceil(self.df.shape[0] / self.batch_size)) #SOMA

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])

        if self.augmentation is not None:
            image = self.augmentation.flow(image, shuffle=False).next()

        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [6]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=42)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [7]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
10416,resized/10418.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0
3495,resized/3496.jpg,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0
6950,resized/6951.jpg,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1
5035,resized/5036.jpg,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4410,resized/4411.jpg,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, 
    augmentation=ImageDataGenerator(
        #featurewise_center = True,
        #featurewise_std_normalization = True,
        horizontal_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
    ))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False, 
    # augmentation=ImageDataGenerator(
    #     featurewise_center = True,
    #     featurewise_std_normalization = True,)
    ) #SOMA

In [10]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [11]:
input = Input(shape=(224, 224, 3,))

conv1 = Conv2D(32, (3,3), strides=(1,1), padding='valid', kernel_regularizer=l2(1.e-4), name='conv_1')(input)										#224,			3
bn1 = BatchNormalization(name='norm_1')(conv1)
act1 = Activation('relu')(bn1)
conv2 = Conv2D(32, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_2')(act1) 										#222			5

#layer1
bn2 = BatchNormalization(name='norm_2')(conv2)
act2 = Activation('relu')(bn2)
conv3 = Conv2D(32, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_3')(act1) 										#222			7								
#add
add1 = add([conv2, conv3]) 																	                                                                                #222

bn3 = BatchNormalization(name='norm_3')(add1)
act3 = Activation('relu')(bn3)
conv4 = Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(1.e-4), name='conv_4')(act3)								#222		9								
bn4 = BatchNormalization(name='norm_4')(conv4)  
act4 = Activation('relu')(bn4)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")(act4)											                                          #111			18

#add
shortconv1 = Conv2D(64, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_1')(add1)				#222			20
shortconv11 = Conv2D(64, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_11')(shortconv1) #111			20
add2 = add([shortconv11, pool1])																						                                                                  #111

#layer2
bn5 = BatchNormalization(name='norm_5')(add2)
act5 = Activation('relu')(bn5)
conv5 = Conv2D(128, (3,3), strides=(1,1), padding='valid', kernel_regularizer=l2(1.e-4), name='conv_5')(act5)								#109	    24 *+4
bn6 = BatchNormalization(name='norm_6')(conv5)
act6 = Activation('relu')(bn6)

shortconv2 = Conv2D(128, (3,3), strides=(1,1), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_2')(add2)				#109		  28
#add
add3 = add([shortconv2, act6])																						                                                                  #109

bn7 = BatchNormalization(name='norm_7')(add3)
act7 = Activation('relu')(bn7)
conv6 = SeparableConv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform', bias_initializer='zeros', depthwise_regularizer=l2(1.e-4), pointwise_regularizer=None, bias_regularizer=None, activity_regularizer=None, depthwise_constraint=None, pointwise_constraint=None, bias_constraint=None)(act7)
bn8 = BatchNormalization(name='norm_8')(conv6)																		                                                          #109			32
act8 = Activation('relu')(bn8)
conv7 = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_regularizer=l2(1.e-4), name='conv_7')(act8)										#55			  36
#add
shortconv3 = Conv2D(256, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_3')(add3)	      #109			  44 *+8
shortconv33 = Conv2D(256, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_33')(shortconv3) #55			44
add4 = add([shortconv33, conv7])																						                                                                  #55


bn9 = BatchNormalization(name='norm_9')(add4)
act9 = Activation('relu')(bn9)
conv8 = SeparableConv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform', bias_initializer='zeros', depthwise_regularizer=l2(1.e-4), pointwise_regularizer=None, bias_regularizer=None, activity_regularizer=None, depthwise_constraint=None, pointwise_constraint=None, bias_constraint=None)(act9)
bn10 = BatchNormalization(name='norm_10')(conv8)																		                                                        #55			  60 *+16
act10 = Activation('relu')(bn10)
conv9 = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_regularizer=l2(1.e-4), name='conv_9')(act10)									#28			  76
#add
shortconv4 = Conv2D(256, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_4')(add4)			#55			  100 *+24
shortconv44 = Conv2D(256, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_44')(shortconv4) #28			100
add5 = add([shortconv44, conv9])																						                                                                  #28

bn11 = BatchNormalization(name='norm_11')(add5)
act11 = Activation('relu')(bn11)
conv10 = SeparableConv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform', bias_initializer='zeros', depthwise_regularizer=l2(1.e-4), pointwise_regularizer=None, bias_regularizer=None, activity_regularizer=None, depthwise_constraint=None, pointwise_constraint=None, bias_constraint=None)(act11)
bn12 = BatchNormalization(name='norm_12')(conv10)																		                                                        #28			  60 *+16
act12 = Activation('relu')(bn12)
conv11 = Conv2D(512, (3,3), strides=(2,2), padding='same', kernel_regularizer=l2(1.e-4), name='conv_11')(act12)									#14			  76
#add
shortconv5 = Conv2D(512, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_5')(add5)			#28			  100 *+24
shortconv55 = Conv2D(512, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_55')(shortconv5) #14			100
add6 = add([shortconv55, conv11])																						                                                                  #14

bn13 = BatchNormalization(name='norm_13')(add6)
act12 = Activation('relu')(bn13)
conv12 = SeparableConv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu', depthwise_initializer='glorot_uniform', pointwise_initializer='glorot_uniform', bias_initializer='zeros', depthwise_regularizer=l2(1.e-4), pointwise_regularizer=None, bias_regularizer=None, activity_regularizer=None, depthwise_constraint=None, pointwise_constraint=None, bias_constraint=None)(act12)
bn14 = BatchNormalization(name='norm_14')(conv12)																		                                                        #14			  60 *+16
act13 = Activation('relu')(bn14)
conv13 = Conv2D(512, (3,3), strides=(2,2), padding='same', kernel_regularizer=l2(1.e-4), name='conv_13')(act13)									#7			  76
#add
shortconv6 = Conv2D(512, (3,3), strides=(1,1), padding='same', kernel_regularizer=l2(0.0001), name='shortconv_6')(add6)			#14			  100 *+24
shortconv66 = Conv2D(512, (1,1), strides=(2,2), padding='valid', kernel_regularizer=l2(0.0001), name='shortconv_66')(shortconv6) #7			100
add7 = add([shortconv66, conv13])

bn15 = BatchNormalization(name='norm_15')(add7)																		                                                          #7			  164  
act11 = Activation('relu')(bn15)

pool2 = GlobalAveragePooling2D()(act11)

def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", pool2)
image_quality = build_head("image_quality", pool2)
age = build_head("age", pool2)
weight = build_head("weight", pool2)
bag = build_head("bag", pool2)
footwear = build_head("footwear", pool2)
emotion = build_head("emotion", pool2)
pose = build_head("pose", pool2)

model = Model(
    inputs=[input], 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 222, 222, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 222, 222, 32) 128         conv_1[0][0]                     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 222, 222, 32) 0           norm_1[0][0]                     
____________________________________________________________________________________________

In [12]:
losses = {
	"gender_output": "binary_crossentropy",
	"image_quality_output": "categorical_crossentropy",
	"age_output": "categorical_crossentropy",
	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy",
}
loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0, "weight_output": 1.0, "bag_output": 1.0, "footwear_output": 1.0, "pose_output": 1.0, "emotion_output": 1.0}
opt = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(
    optimizer=opt,
    loss=losses, #"categorical_crossentropy",
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-7)
filepath="/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=30,
    verbose=1,
    callbacks=[checkpoint]
)



Epoch 1/30
361/361 [==============================] - 244s 676ms/step - loss: 8.4255 - gender_output_loss: 0.6883 - image_quality_output_loss: 0.9947 - age_output_loss: 1.4514 - weight_output_loss: 1.0117 - bag_output_loss: 0.9341 - footwear_output_loss: 0.9944 - pose_output_loss: 0.9488 - emotion_output_loss: 0.9287 - gender_output_acc: 0.5597 - image_quality_output_acc: 0.5413 - age_output_acc: 0.3862 - weight_output_acc: 0.6272 - bag_output_acc: 0.5510 - footwear_output_acc: 0.5218 - pose_output_acc: 0.6119 - emotion_output_acc: 0.7093 - val_loss: 8.8843 - val_gender_output_loss: 0.7191 - val_image_quality_output_loss: 0.9991 - val_age_output_loss: 1.6092 - val_weight_output_loss: 1.0287 - val_bag_output_loss: 1.0002 - val_footwear_output_loss: 1.0886 - val_pose_output_loss: 0.9596 - val_emotion_output_loss: 1.0094 - val_gender_output_acc: 0.4573 - val_image_quality_output_acc: 0.5653 - val_age_output_acc: 0.3880 - val_weight_output_acc: 0.6469 - val_bag_output_acc: 0.5560 - val_f

In [0]:
#model.save_weights('/content/gdrive/My Drive/EIP_Assignment5_v2.hdf5')

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec.hdf5')

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, 
    augmentation=ImageDataGenerator(
        #featurewise_center = True,
        #featurewise_std_normalization = True,
        horizontal_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
    ))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False, 
    # augmentation=ImageDataGenerator(
    #     featurewise_center = True,
    #     featurewise_std_normalization = True,)
    ) #SOMA

In [0]:
from keras import backend as K
K.set_value(model.optimizer.lr, 1e-2)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,
    callbacks=[checkpoint, lr_reducer]
)

Epoch 1/25
361/361 [==============================] - 225s 622ms/step - loss: 6.6614 - gender_output_loss: 0.3263 - image_quality_output_loss: 0.8661 - age_output_loss: 1.3127 - weight_output_loss: 0.9214 - bag_output_loss: 0.7865 - footwear_output_loss: 0.7766 - pose_output_loss: 0.4517 - emotion_output_loss: 0.8581 - gender_output_acc: 0.8637 - image_quality_output_acc: 0.5828 - age_output_acc: 0.4192 - weight_output_acc: 0.6406 - bag_output_acc: 0.6656 - footwear_output_acc: 0.6574 - pose_output_acc: 0.8207 - emotion_output_acc: 0.7126 - val_loss: 8.1710 - val_gender_output_loss: 0.4869 - val_image_quality_output_loss: 1.0674 - val_age_output_loss: 1.3993 - val_weight_output_loss: 0.9700 - val_bag_output_loss: 0.8483 - val_footwear_output_loss: 1.2431 - val_pose_output_loss: 0.8624 - val_emotion_output_loss: 0.9325 - val_gender_output_acc: 0.7957 - val_image_quality_output_acc: 0.4528 - val_age_output_acc: 0.3890 - val_weight_output_acc: 0.6464 - val_bag_output_acc: 0.6341 - val_foo

In [0]:
seq = iaa.Sequential(
      [
      iaa.Sometimes(0.5, iaa.CoarseDropout((0.5), size_percent=(0.01))),
		  #iaa.Sometimes(0.5, iaa.Crop(percent=(0, 0.6))),
		  #iaa.Sometimes(0.4, iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, rotate=(-30, 30))),	  
      #iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5))),
      # iaa.Sometimes(0.1, iaa.OneOf([
      #               iaa.EdgeDetect(alpha=(0, 0.7)),
      #               iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0)),
      #               ])),
      iaa.Sometimes(0.4, iaa.AdditiveGaussianNoise(scale=(0.1))),
	  ],

      # do all of the above augmentations in random order
      random_order=True
  )

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, 
    augmentation=ImageDataGenerator(
        #featurewise_center = True,
        #featurewise_std_normalization = True,
        preprocessing_function = seq.augment_image
    ))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False, 
    # augmentation=ImageDataGenerator(
    #     featurewise_center = True,
    #     featurewise_std_normalization = True,)
    ) #SOMA

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec.hdf5') #loss = 6.35742

In [0]:
from keras import backend as K
K.set_value(model.optimizer.lr, 1e-2)

In [0]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-7)
filepath="/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v1.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=25,
    verbose=1,
    callbacks=[checkpoint, lr_reducer]
)

Epoch 1/25
361/361 [==============================] - 221s 614ms/step - loss: 6.3245 - gender_output_loss: 0.2938 - image_quality_output_loss: 0.8445 - age_output_loss: 1.2139 - weight_output_loss: 0.8556 - bag_output_loss: 0.7245 - footwear_output_loss: 0.7536 - pose_output_loss: 0.4523 - emotion_output_loss: 0.8402 - gender_output_acc: 0.8685 - image_quality_output_acc: 0.6016 - age_output_acc: 0.4636 - weight_output_acc: 0.6599 - bag_output_acc: 0.6972 - footwear_output_acc: 0.6675 - pose_output_acc: 0.8171 - emotion_output_acc: 0.7129 - val_loss: 7.3422 - val_gender_output_loss: 0.3505 - val_image_quality_output_loss: 0.9965 - val_age_output_loss: 1.4135 - val_weight_output_loss: 0.9254 - val_bag_output_loss: 1.0982 - val_footwear_output_loss: 0.7635 - val_pose_output_loss: 0.5337 - val_emotion_output_loss: 0.9146 - val_gender_output_acc: 0.8492 - val_image_quality_output_acc: 0.5580 - val_age_output_acc: 0.4077 - val_weight_output_acc: 0.6508 - val_bag_output_acc: 0.5815 - val_foo

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v1.hdf5') #loss = 7.34217

In [0]:
def evaluate_model(model):
    results = model.evaluate_generator(valid_gen, verbose=1)
    accuracies = {}
    losses = {}
    for k, v in zip(model.metrics_names, results):
        if k.endswith('acc'):
            accuracies[k] = round(v * 100, 4) 
        else:
            losses[k] = v
    return accuracies

In [0]:
evaluate_model(model)

32/32 [==============================] - 20s 620ms/step


{'age_output_acc': 43.3694,
 'bag_output_acc': 63.998,
 'emotion_output_acc': 69.7446,
 'footwear_output_acc': 64.5874,
 'gender_output_acc': 81.336,
 'image_quality_output_acc': 58.6444,
 'pose_output_acc': 83.5462,
 'weight_output_acc': 62.0825}

In [0]:
results = model.evaluate_generator(valid_gen, verbose=1)

dict(zip(model.metrics_names, results))

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec.hdf5') #loss = 6.35742

In [0]:
evaluate_model(model)

32/32 [==============================] - 11s 333ms/step


{'age_output_acc': 44.3517,
 'bag_output_acc': 70.334,
 'emotion_output_acc': 70.0884,
 'footwear_output_acc': 67.2888,
 'gender_output_acc': 90.9627,
 'image_quality_output_acc': 60.6582,
 'pose_output_acc': 84.7741,
 'weight_output_acc': 64.833}

In [0]:
results = model.evaluate_generator(valid_gen, verbose=1)

dict(zip(model.metrics_names, results))

In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, 
    augmentation=ImageDataGenerator(
        #featurewise_center = True,
        #featurewise_std_normalization = True,
        horizontal_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
    ))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False, 
    # augmentation=ImageDataGenerator(
    #     featurewise_center = True,
    #     featurewise_std_normalization = True,)
    ) #SOMA

In [0]:
def lr_schedule(epoch):
    if epoch > 140:
        lr = 1e-4
    elif epoch > 105:
        lr = 5e-4
    elif epoch > 70:
        lr = 5e-3
    elif epoch > 45:
        lr = 5e-2
    elif epoch > 30:
        lr = 1e-2
    elif epoch > 15:
        lr = 5e-2
    else:
        lr = 1e-1
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

In [0]:
#lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.001, cooldown=0, min_lr=1e-4)
filepath="/content/gdrive/My Drive/EIP_Assignment5_v2_25Dec_v2.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v1.hdf5')

In [0]:
from keras import backend as K
K.set_value(model.optimizer.lr, 1e-1)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=150,
    verbose=1,
    callbacks=[checkpoint, lr_scheduler]
)

Epoch 1/150
Learning rate (from LearningRateScheduler):  0.1
360/361 [============================>.] - ETA: 0s - loss: 8.2834 - gender_output_loss: 0.6951 - image_quality_output_loss: 0.9944 - age_output_loss: 1.4384 - weight_output_loss: 1.0000 - bag_output_loss: 0.9323 - footwear_output_loss: 0.9989 - pose_output_loss: 0.9329 - emotion_output_loss: 0.9116 - gender_output_acc: 0.5577 - image_quality_output_acc: 0.5468 - age_output_acc: 0.3930 - weight_output_acc: 0.6283 - bag_output_acc: 0.5550 - footwear_output_acc: 0.5145 - pose_output_acc: 0.6181 - emotion_output_acc: 0.7128 0.1

 0.1
361/361 [==============================] - 224s 620ms/step - loss: 8.2827 - gender_output_loss: 0.6951 - image_quality_output_loss: 0.9941 - age_output_loss: 1.4384 - weight_output_loss: 0.9998 - bag_output_loss: 0.9322 - footwear_output_loss: 0.9987 - pose_output_loss: 0.9328 - emotion_output_loss: 0.9118 - gender_output_acc: 0.5577 - image_quality_output_acc: 0.5471 - age_output_acc: 0.3931 - weigh

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v2_25Dec_v2.hdf5') # loss = 6.48211

In [0]:
from keras import backend as K
K.set_value(model.optimizer.lr, 5e-2)

In [0]:
filepath="/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v3.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=5,
    verbose=1,
    callbacks=[checkpoint]
)



Epoch 1/5
361/361 [==============================] - 244s 676ms/step - loss: 6.6127 - gender_output_loss: 0.3626 - image_quality_output_loss: 0.8828 - age_output_loss: 1.3362 - weight_output_loss: 0.9291 - bag_output_loss: 0.8009 - footwear_output_loss: 0.7960 - pose_output_loss: 0.4798 - emotion_output_loss: 0.8644 - gender_output_acc: 0.8392 - image_quality_output_acc: 0.5802 - age_output_acc: 0.4186 - weight_output_acc: 0.6378 - bag_output_acc: 0.6504 - footwear_output_acc: 0.6446 - pose_output_acc: 0.8129 - emotion_output_acc: 0.7110 - val_loss: 9.8590 - val_gender_output_loss: 0.8286 - val_image_quality_output_loss: 1.1559 - val_age_output_loss: 1.5917 - val_weight_output_loss: 0.9851 - val_bag_output_loss: 0.9377 - val_footwear_output_loss: 2.3016 - val_pose_output_loss: 0.8955 - val_emotion_output_loss: 0.9887 - val_gender_output_acc: 0.6807 - val_image_quality_output_acc: 0.4425 - val_age_output_acc: 0.3335 - val_weight_output_acc: 0.6267 - val_bag_output_acc: 0.5712 - val_fo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1,
    callbacks=[checkpoint]
)

Epoch 1/10
361/361 [==============================] - 224s 622ms/step - loss: 6.4613 - gender_output_loss: 0.3156 - image_quality_output_loss: 0.8752 - age_output_loss: 1.2929 - weight_output_loss: 0.8974 - bag_output_loss: 0.7744 - footwear_output_loss: 0.7684 - pose_output_loss: 0.4292 - emotion_output_loss: 0.8558 - gender_output_acc: 0.8652 - image_quality_output_acc: 0.5805 - age_output_acc: 0.4323 - weight_output_acc: 0.6465 - bag_output_acc: 0.6690 - footwear_output_acc: 0.6607 - pose_output_acc: 0.8333 - emotion_output_acc: 0.7120 - val_loss: 7.8656 - val_gender_output_loss: 0.3569 - val_image_quality_output_loss: 1.6425 - val_age_output_loss: 1.3454 - val_weight_output_loss: 0.9371 - val_bag_output_loss: 0.8450 - val_footwear_output_loss: 0.8495 - val_pose_output_loss: 0.7221 - val_emotion_output_loss: 0.9077 - val_gender_output_acc: 0.8399 - val_image_quality_output_acc: 0.3168 - val_age_output_acc: 0.4018 - val_weight_output_acc: 0.6478 - val_bag_output_acc: 0.6144 - val_foo

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v3.hdf5') #loss = 7.25337

In [0]:
def lr_schedule(epoch):
    if epoch > 45:
        lr = 1e-4
    elif epoch > 30:
        lr = 5e-4
    elif epoch > 15:
        lr = 1e-3
    else:
        lr = 5e-3
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

In [0]:
filepath="/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v4.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=60,
    verbose=1,
    callbacks=[checkpoint, lr_scheduler]
)

Epoch 1/60
Learning rate (from LearningRateScheduler):  0.005
360/361 [============================>.] - ETA: 0s - loss: 6.1594 - gender_output_loss: 0.2588 - image_quality_output_loss: 0.8481 - age_output_loss: 1.2539 - weight_output_loss: 0.8732 - bag_output_loss: 0.7369 - footwear_output_loss: 0.7429 - pose_output_loss: 0.3727 - emotion_output_loss: 0.8437 - gender_output_acc: 0.8965 - image_quality_output_acc: 0.5990 - age_output_acc: 0.4450 - weight_output_acc: 0.6553 - bag_output_acc: 0.6918 - footwear_output_acc: 0.6717 - pose_output_acc: 0.8614 - emotion_output_acc: 0.7126Learning rate (from LearningRateScheduler):  0.005
Learning rate (from LearningRateScheduler):  0.005
361/361 [==============================] - 223s 619ms/step - loss: 6.1588 - gender_output_loss: 0.2585 - image_quality_output_loss: 0.8476 - age_output_loss: 1.2541 - weight_output_loss: 0.8733 - bag_output_loss: 0.7370 - footwear_output_loss: 0.7429 - pose_output_loss: 0.3722 - emotion_output_loss: 0.8439 - g

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v4.hdf5') # loss = 6.27273

In [0]:
evaluate_model(model)

32/32 [==============================] - 11s 333ms/step


{'age_output_acc': 44.4499,
 'bag_output_acc': 69.2534,
 'emotion_output_acc': 70.6778,
 'footwear_output_acc': 65.0295,
 'gender_output_acc': 90.4715,
 'image_quality_output_acc': 59.5776,
 'pose_output_acc': 85.609,
 'weight_output_acc': 65.7171}

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v3.hdf5') # loss = 7.25337

In [0]:
evaluate_model(model)

32/32 [==============================] - 11s 333ms/step


{'age_output_acc': 41.4538,
 'bag_output_acc': 60.0688,
 'emotion_output_acc': 70.5305,
 'footwear_output_acc': 52.2102,
 'gender_output_acc': 77.5049,
 'image_quality_output_acc': 57.3183,
 'pose_output_acc': 75.2456,
 'weight_output_acc': 64.7839}

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v4.hdf5') # loss = 6.27273

In [0]:
!wget https://raw.githubusercontent.com/yu4u/cutout-random-erasing/master/random_eraser.py

from random_eraser import get_random_eraser

--2019-12-27 04:45:23--  https://raw.githubusercontent.com/yu4u/cutout-random-erasing/master/random_eraser.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 888 [text/plain]
Saving to: ‘random_eraser.py’

random_eraser.py    100%[===================>]     888  --.-KB/s    in 0s      

2019-12-27 04:45:23 (293 MB/s) - ‘random_eraser.py’ saved [888/888]



In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32, 
    augmentation=ImageDataGenerator(
        #featurewise_center = True,
        #featurewise_std_normalization = True,
        horizontal_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
    ))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False, 
    # augmentation=ImageDataGenerator(
    #     featurewise_center = True,
    #     featurewise_std_normalization = True,)
    ) #SOMA

In [0]:
def lr_schedule(epoch):
    if epoch > 40:
        lr = 1e-4
    elif epoch > 30:
        lr = 5e-4
    elif epoch > 20:
        lr = 1e-3
    elif epoch > 10:
        lr = 5e-3
    else:
        lr = 1e-2
    print('Learning rate (from LearningRateScheduler): ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

In [0]:
filepath="/content/gdrive/My Drive/EIP_Assignment5_v3_27Dec_v5.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks=[checkpoint, lr_scheduler]
)



Epoch 1/50
Learning rate (from LearningRateScheduler):  0.01
360/361 [============================>.] - ETA: 0s - loss: 6.1505 - gender_output_loss: 0.2606 - image_quality_output_loss: 0.8483 - age_output_loss: 1.2370 - weight_output_loss: 0.8643 - bag_output_loss: 0.7319 - footwear_output_loss: 0.7500 - pose_output_loss: 0.3925 - emotion_output_loss: 0.8419 - gender_output_acc: 0.8849 - image_quality_output_acc: 0.5978 - age_output_acc: 0.4569 - weight_output_acc: 0.6578 - bag_output_acc: 0.6936 - footwear_output_acc: 0.6661 - pose_output_acc: 0.8493 - emotion_output_acc: 0.7130

361/361 [==============================] - 240s 665ms/step - loss: 6.1515 - gender_output_loss: 0.2610 - image_quality_output_loss: 0.8486 - age_output_loss: 1.2371 - weight_output_loss: 0.8640 - bag_output_loss: 0.7320 - footwear_output_loss: 0.7506 - pose_output_loss: 0.3923 - emotion_output_loss: 0.8418 - gender_output_acc: 0.8849 - image_quality_output_acc: 0.5978 - age_output_acc: 0.4566 - weight_outpu

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_27Dec_v5.hdf5') # loss = 6.45580

In [0]:
import os
import numpy as np
import warnings

from keras.callbacks import Callback
from keras import backend as K

import numpy as np
import scipy.ndimage.filters as filters
import scipy.ndimage.morphology as morphology
import pandas as pd


class LRFinder(Callback):
    def __init__(self,
                 num_samples,
                 batch_size,
                 minimum_lr=1e-5,
                 maximum_lr=10.,
                 lr_scale='exp',
                 validation_data=None,
                 validation_sample_rate=5,
                 stopping_criterion_factor=4.,
                 loss_smoothing_beta=0.98,
                 save_dir=None,
                 verbose=True):
        """
        This class uses the Cyclic Learning Rate history to find a
        set of learning rates that can be good initializations for the
        One-Cycle training proposed by Leslie Smith in the paper referenced
        below.
        A port of the Fast.ai implementation for Keras.
        # Note
        This requires that the model be trained for exactly 1 epoch. If the model
        is trained for more epochs, then the metric calculations are only done for
        the first epoch.
        # Interpretation
        Upon visualizing the loss plot, check where the loss starts to increase
        rapidly. Choose a learning rate at somewhat prior to the corresponding
        position in the plot for faster convergence. This will be the maximum_lr lr.
        Choose the max value as this value when passing the `max_val` argument
        to OneCycleLR callback.
        Since the plot is in log-scale, you need to compute 10 ^ (-k) of the x-axis
        # Arguments:
            num_samples: Integer. Number of samples in the dataset.
            batch_size: Integer. Batch size during training.
            minimum_lr: Float. Initial learning rate (and the minimum).
            maximum_lr: Float. Final learning rate (and the maximum).
            lr_scale: Can be one of ['exp', 'linear']. Chooses the type of
                scaling for each update to the learning rate during subsequent
                batches. Choose 'exp' for large range and 'linear' for small range.
            validation_data: Requires the validation dataset as a tuple of
                (X, y) belonging to the validation set. If provided, will use the
                validation set to compute the loss metrics. Else uses the training
                batch loss. Will warn if not provided to alert the user.
            validation_sample_rate: Positive or Negative Integer. Number of batches to sample from the
                validation set per iteration of the LRFinder. Larger number of
                samples will reduce the variance but will take longer time to execute
                per batch.
                If Positive > 0, will sample from the validation dataset
                If Megative, will use the entire dataset
            stopping_criterion_factor: Integer or None. A factor which is used
                to measure large increase in the loss value during training.
                Since callbacks cannot stop training of a model, it will simply
                stop logging the additional values from the epochs after this
                stopping criterion has been met.
                If None, this check will not be performed.
            loss_smoothing_beta: Float. The smoothing factor for the moving
                average of the loss function.
            save_dir: Optional, String. If passed a directory path, the callback
                will save the running loss and learning rates to two separate numpy
                arrays inside this directory. If the directory in this path does not
                exist, they will be created.
            verbose: Whether to print the learning rate after every batch of training.
        # References:
            - [A disciplined approach to neural network hyper-parameters: Part 1 -- learning rate, batch size, weight_decay, and weight decay](https://arxiv.org/abs/1803.09820)
        """
        super(LRFinder, self).__init__()

        if lr_scale not in ['exp', 'linear']:
            raise ValueError("`lr_scale` must be one of ['exp', 'linear']")

        if validation_data is not None:
            self.validation_data = validation_data
            self.use_validation_set = True

            if validation_sample_rate > 0 or validation_sample_rate < 0:
                self.validation_sample_rate = validation_sample_rate
            else:
                raise ValueError("`validation_sample_rate` must be a positive or negative integer other than o")
        else:
            self.use_validation_set = False
            self.validation_sample_rate = 0

        self.num_samples = num_samples
        self.batch_size = batch_size
        self.initial_lr = minimum_lr
        self.final_lr = maximum_lr
        self.lr_scale = lr_scale
        self.stopping_criterion_factor = stopping_criterion_factor
        self.loss_smoothing_beta = loss_smoothing_beta
        self.save_dir = save_dir
        self.verbose = verbose

        self.num_batches_ = num_samples // batch_size
        self.current_lr_ = minimum_lr

        if lr_scale == 'exp':
            self.lr_multiplier_ = (maximum_lr / float(minimum_lr)) ** (
                1. / float(self.num_batches_))
        else:
            extra_batch = int((num_samples % batch_size) != 0)
            self.lr_multiplier_ = np.linspace(
                minimum_lr, maximum_lr, num=self.num_batches_ + extra_batch)

        # If negative, use entire validation set
        if self.validation_sample_rate < 0:
            self.validation_sample_rate = self.validation_data[0].shape[0] // batch_size

        self.current_batch_ = 0
        self.current_epoch_ = 0
        self.best_loss_ = 1e6
        self.running_loss_ = 0.

        self.history = {}

    def on_train_begin(self, logs=None):

        self.current_epoch_ = 1
        K.set_value(self.model.optimizer.lr, self.initial_lr)

        warnings.simplefilter("ignore")

    def on_epoch_begin(self, epoch, logs=None):
        self.current_batch_ = 0

        if self.current_epoch_ > 1:
            warnings.warn(
                "\n\nLearning rate finder should be used only with a single epoch. "
                "Hereafter, the callback will not measure the losses.\n\n")

    def on_batch_begin(self, batch, logs=None):
        self.current_batch_ += 1

    def on_batch_end(self, batch, logs=None):
        if self.current_epoch_ > 1:
            return

        if self.use_validation_set:
            X, Y = self.validation_data[0], self.validation_data[1]

            # use 5 random batches from test set for fast approximate of loss
            num_samples = self.batch_size * self.validation_sample_rate

            if num_samples > X.shape[0]:
                num_samples = X.shape[0]

            idx = np.random.choice(X.shape[0], num_samples, replace=False)
            x = X[idx]
            y = Y[idx]

            values = self.model.evaluate(x, y, batch_size=self.batch_size, verbose=False)
            loss = values[0]
        else:
            loss = logs['loss']

        # smooth the loss value and bias correct
        running_loss = self.loss_smoothing_beta * loss + (
            1. - self.loss_smoothing_beta) * loss
        running_loss = running_loss / (
            1. - self.loss_smoothing_beta**self.current_batch_)

        # stop logging if loss is too large
        if self.current_batch_ > 1 and self.stopping_criterion_factor is not None and (
                running_loss >
                self.stopping_criterion_factor * self.best_loss_):

            if self.verbose:
                print(" - LRFinder: Skipping iteration since loss is %d times as large as best loss (%0.4f)"
                      % (self.stopping_criterion_factor, self.best_loss_))
            return

        if running_loss < self.best_loss_ or self.current_batch_ == 1:
            self.best_loss_ = running_loss

        current_lr = K.get_value(self.model.optimizer.lr)

        self.history.setdefault('running_loss_', []).append(running_loss)
        if self.lr_scale == 'exp':
            self.history.setdefault('log_lrs', []).append(np.log10(current_lr))
        else:
            self.history.setdefault('log_lrs', []).append(current_lr)

        # compute the lr for the next batch and update the optimizer lr
        if self.lr_scale == 'exp':
            current_lr *= self.lr_multiplier_
        else:
            current_lr = self.lr_multiplier_[self.current_batch_ - 1]

        K.set_value(self.model.optimizer.lr, current_lr)

        # save the other metrics as well
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        if self.verbose:
            if self.use_validation_set:
                print(" - LRFinder: val_loss: %1.4f - lr = %1.8f " %
                      (values[0], current_lr))
            else:
                print(" - LRFinder: lr = %1.8f " % current_lr)

    def on_epoch_end(self, epoch, logs=None):
        if self.save_dir is not None and self.current_epoch_ <= 1:
            if not os.path.exists(self.save_dir):
                os.makedirs(self.save_dir)

            losses_path = os.path.join(self.save_dir, 'losses.npy')
            lrs_path = os.path.join(self.save_dir, 'lrs.npy')

            np.save(losses_path, self.losses)
            np.save(lrs_path, self.lrs)

            if self.verbose:
                print("\tLR Finder : Saved the losses and learning rate values in path : {%s}"
                      % (self.save_dir))

        self.current_epoch_ += 1

        warnings.simplefilter("default")

    def plot_schedule(self, clip_beginning=None, clip_endding=None):
        """
        Plots the schedule from the callback itself.
        # Arguments:
            clip_beginning: Integer or None. If positive integer, it will
                remove the specified portion of the loss graph to remove the large
                loss values in the beginning of the graph.
            clip_endding: Integer or None. If negative integer, it will
                remove the specified portion of the ending of the loss graph to
                remove the sharp increase in the loss values at high learning rates.
        """
        try:
            import matplotlib.pyplot as plt
            plt.style.use('seaborn-white')
        except ImportError:
            print(
                "Matplotlib not found. Please use `pip install matplotlib` first."
            )
            return

        if clip_beginning is not None and clip_beginning < 0:
            clip_beginning = -clip_beginning

        if clip_endding is not None and clip_endding > 0:
            clip_endding = -clip_endding

        losses = self.losses
        lrs = self.lrs

        if clip_beginning:
            losses = losses[clip_beginning:]
            lrs = lrs[clip_beginning:]

        if clip_endding:
            losses = losses[:clip_endding]
            lrs = lrs[:clip_endding]

        plt.plot(lrs, losses)
        plt.title('Learning rate vs Loss')
        plt.xlabel('learning rate')
        plt.ylabel('loss')
        plt.show()

    @classmethod
    def restore_schedule_from_dir(cls,
                                  directory,
                                  clip_beginning=None,
                                  clip_endding=None):
        """
        Loads the training history from the saved numpy files in the given directory.
        # Arguments:
            directory: String. Path to the directory where the serialized numpy
                arrays of the loss and learning rates are saved.
            clip_beginning: Integer or None. If positive integer, it will
                remove the specified portion of the loss graph to remove the large
                loss values in the beginning of the graph.
            clip_endding: Integer or None. If negative integer, it will
                remove the specified portion of the ending of the loss graph to
                remove the sharp increase in the loss values at high learning rates.
        Returns:
            tuple of (losses, learning rates)
        """
        if clip_beginning is not None and clip_beginning < 0:
            clip_beginning = -clip_beginning

        if clip_endding is not None and clip_endding > 0:
            clip_endding = -clip_endding

        losses_path = os.path.join(directory, 'losses.npy')
        lrs_path = os.path.join(directory, 'lrs.npy')

        if not os.path.exists(losses_path) or not os.path.exists(lrs_path):
            print("%s and %s could not be found at directory : {%s}" %
                  (losses_path, lrs_path, directory))

            losses = None
            lrs = None

        else:
            losses = np.load(losses_path)
            lrs = np.load(lrs_path)

            if clip_beginning:
                losses = losses[clip_beginning:]
                lrs = lrs[clip_beginning:]

            if clip_endding:
                losses = losses[:clip_endding]
                lrs = lrs[:clip_endding]

        return losses, lrs

    @classmethod
    def plot_schedule_from_file(cls,
                                directory,
                                clip_beginning=None,
                                clip_endding=None):
        """
        Plots the schedule from the saved numpy arrays of the loss and learning
        rate values in the specified directory.
        # Arguments:
            directory: String. Path to the directory where the serialized numpy
                arrays of the loss and learning rates are saved.
            clip_beginning: Integer or None. If positive integer, it will
                remove the specified portion of the loss graph to remove the large
                loss values in the beginning of the graph.
            clip_endding: Integer or None. If negative integer, it will
                remove the specified portion of the ending of the loss graph to
                remove the sharp increase in the loss values at high learning rates.
        """
        try:
            import matplotlib.pyplot as plt
            plt.style.use('seaborn-white')
        except ImportError:
            print("Matplotlib not found. Please use `pip install matplotlib` first.")
            return

        losses, lrs = cls.restore_schedule_from_dir(
            directory,
            clip_beginning=clip_beginning,
            clip_endding=clip_endding)

        if losses is None or lrs is None:
            return
        else:
            plt.plot(lrs, losses)
            plt.title('Learning rate vs Loss')
            plt.xlabel('learning rate')
            plt.ylabel('loss')
            plt.show()

    @property
    def lrs(self):
        return np.array(self.history['log_lrs'])

    @property
    def losses(self):
        return np.array(self.history['running_loss_'])

In [0]:
import os
import numpy as np
import warnings

from keras.callbacks import Callback
from keras import backend as K


# Code is ported from https://github.com/fastai/fastai
class OneCycleLR(Callback):
    def __init__(self,
                 epochs,
                 batch_size,
                 samples,
                 max_lr,
                 end_percentage=0.1,
                 scale=100,
                 maximum_momentum=0.95,
                 minimum_momentum=0.85,
                 verbose=True):
        """ This callback implements a cyclical learning rate policy (CLR).
        This is a special case of Cyclic Learning Rates, where we have only 1 cycle.
        After the completion of 1 cycle, the learning rate will decrease rapidly to
        100th its initial lowest value.
        # Arguments:
            max_lr: Float. Initial learning rate. This also sets the
                starting learning rate (which will be 10x smaller than
                this), and will increase to this value during the first cycle.
            end_percentage: Float. The percentage of all the epochs of training
                that will be dedicated to sharply decreasing the learning
                rate after the completion of 1 cycle. Must be between 0 and 1.
            scale_percentage: Float or None. If float, must be between 0 and 1.
                If None, it will compute the scale_percentage automatically
                based on the `end_percentage`.
            maximum_momentum: Optional. Sets the maximum momentum (initial)
                value, which gradually drops to its lowest value in half-cycle,
                then gradually increases again to stay constant at this max value.
                Can only be used with SGD Optimizer.
            minimum_momentum: Optional. Sets the minimum momentum at the end of
                the half-cycle. Can only be used with SGD Optimizer.
            verbose: Bool. Whether to print the current learning rate after every
                epoch.
        # Reference
            - [A disciplined approach to neural network hyper-parameters: Part 1 -- learning rate, batch size, weight_decay, and weight decay](https://arxiv.org/abs/1803.09820)
            - [Super-Convergence: Very Fast Training of Residual Networks Using Large Learning Rates](https://arxiv.org/abs/1708.07120)
        """
        super(OneCycleLR, self).__init__()

        if end_percentage < 0. or end_percentage > 1.:
            raise ValueError("`end_percentage` must be between 0 and 1")


        self.initial_lr = max_lr
        self.end_percentage = end_percentage
        self.scale = scale
        self.max_momentum = maximum_momentum
        self.min_momentum = minimum_momentum
        self.verbose = verbose

        if self.max_momentum is not None and self.min_momentum is not None:
            self._update_momentum = True
        else:
            self._update_momentum = False

        self.clr_iterations = 0.
        self.history = {}

        self.epochs = epochs
        self.batch_size = batch_size
        self.samples = samples
        self.steps = None
        self.num_iterations = None
        self.mid_cycle_id = None

    def _reset(self):
        """
        Reset the callback.
        """
        self.clr_iterations = 0.
        self.history = {}

    def compute_lr(self):
        """
        Compute the learning rate based on which phase of the cycle it is in.
        - If in the first half of training, the learning rate gradually increases.
        - If in the second half of training, the learning rate gradually decreases.
        - If in the final `end_percentage` portion of training, the learning rate
            is quickly reduced to near 100th of the original min learning rate.
        # Returns:
            the new learning rate
        """
        if self.clr_iterations > 2 * self.mid_cycle_id:
            current_percentage = (self.clr_iterations - 2 * self.mid_cycle_id)
            current_percentage /= float((self.num_iterations - 2 * self.mid_cycle_id))
            new_lr = self.initial_lr * (1. + (current_percentage * (1. - 100.) / 100.)) / self.scale

        elif self.clr_iterations > self.mid_cycle_id:
            current_percentage = 1. - (
                    self.clr_iterations - self.mid_cycle_id) / self.mid_cycle_id
            new_lr = self.initial_lr * (1. + current_percentage * (self.scale * 100 - 1.)) / self.scale

        else:
            current_percentage = self.clr_iterations / self.mid_cycle_id
            new_lr = self.initial_lr * (1. + current_percentage * (self.scale - 1.)) / self.scale

        if self.clr_iterations == self.num_iterations:
            self.clr_iterations = 0

        return new_lr

    def compute_momentum(self):
        """
         Compute the momentum based on which phase of the cycle it is in.
        - If in the first half of training, the momentum gradually decreases.
        - If in the second half of training, the momentum gradually increases.
        - If in the final `end_percentage` portion of training, the momentum value
            is kept constant at the maximum initial value.
        # Returns:
            the new momentum value
        """
        if self.clr_iterations > 2 * self.mid_cycle_id:
            new_momentum = self.max_momentum

        elif self.clr_iterations > self.mid_cycle_id:
            current_percentage = 1. - ((self.clr_iterations - self.mid_cycle_id) / float(
                self.mid_cycle_id))
            new_momentum = self.max_momentum - current_percentage * (
                    self.max_momentum - self.min_momentum)

        else:
            current_percentage = self.clr_iterations / float(self.mid_cycle_id)
            new_momentum = self.max_momentum - current_percentage * (
                    self.max_momentum - self.min_momentum)

        return new_momentum

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.steps is not None:
            self.num_iterations = self.epochs * self.steps
        else:
            if (self.samples % self.batch_size) == 0:
                remainder = 0
            else:
                remainder = 1
            self.num_iterations = (self.epochs + remainder) * self.samples // self.batch_size

        self.mid_cycle_id = int(self.num_iterations * ((1. - self.end_percentage)) / float(2))

        self._reset()
        K.set_value(self.model.optimizer.lr, self.compute_lr())

        if self._update_momentum:
            if not hasattr(self.model.optimizer, 'momentum'):
                raise ValueError("Momentum can be updated only on SGD optimizer !")

            new_momentum = self.compute_momentum()
            K.set_value(self.model.optimizer.momentum, new_momentum)

    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}

        self.clr_iterations += 1
        new_lr = self.compute_lr()

        self.history.setdefault('lr', []).append(
            K.get_value(self.model.optimizer.lr))
        K.set_value(self.model.optimizer.lr, new_lr)

        if self._update_momentum:
            if not hasattr(self.model.optimizer, 'momentum'):
                raise ValueError("Momentum can be updated only on SGD optimizer !")

            new_momentum = self.compute_momentum()

            self.history.setdefault('momentum', []).append(
                K.get_value(self.model.optimizer.momentum))
            K.set_value(self.model.optimizer.momentum, new_momentum)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        if self.verbose:
            if self._update_momentum:
                print(" - lr: %0.5f - momentum: %0.2f " %
                      (self.history['lr'][-1], self.history['momentum'][-1]))

            else:
                print(" - lr: %0.5f " % (self.history['lr'][-1]))

In [0]:
nb_epoch = 1

weights_file = 'weights/model.h5'
model_checkpoint = ModelCheckpoint(weights_file, monitor='val_acc', save_best_only=True,
                                   save_weights_only=True, mode='max')

In [0]:
# Exponential lr finder
# USE THIS FOR A LARGE RANGE SEARCH
# Uncomment the validation_data flag to reduce speed but get a better idea of the learning rate
#lr_finder = LRFinder(n_train, BATCH_SIZE, minimum_lr=1e-3, maximum_lr=10.,
                     #lr_scale='exp',
                     #validation_data=(X_test, Y_test),  # use the validation data for losses
                     #validation_sample_rate=5,
                     #save_dir='weights/', verbose=True)

# Linear lr finder
# USE THIS FOR A CLOSE SEARCH
# Uncomment the validation_data flag to reduce speed but get a better idea of the learning rate
lr_finder = LRFinder(13573, 16, minimum_lr=5e-4, maximum_lr=1e-1,
                      lr_scale='linear',
                      validation_data=valid_gen,  # use the validation data for losses
                      validation_sample_rate=5,
                      save_dir='weights/', verbose=True)


# plot the previous values if present
LRFinder.plot_schedule_from_file('weights/', clip_beginning=10, clip_endding=5)

optimizer = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=1, 
    epochs=1,
    verbose=1,
    callbacks=[model_checkpoint]
)

lr_finder.plot_schedule(clip_beginning=10, clip_endding=5)

scores = model.evaluate_generator(valid_gen, verbose=1, batch_size=64) #model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE)
for score, metric_name in zip(scores, model.metrics_names):
    print("%s : %0.4f" % (metric_name, score))

weights/losses.npy and weights/lrs.npy could not be found at directory : {weights/}
Epoch 1/1


ResourceExhaustedError: ignored

In [0]:
MOMENTUMS = [0.9, 0.95, 0.99]
nb_epoch = 1

for momentum in MOMENTUMS:
#     # Learning rate range obtained from `find_lr_schedule.py`
#     # NOTE : Minimum is 10x smaller than the max found above !
#     # NOTE : It is preferable to use the validation data here to get a correct value
      lr_finder = LRFinder(13573, 16, minimum_lr=0.0018, maximum_lr=1e-2,
                          validation_data=(X_test, Y_test),
                          validation_sample_rate=5,
                          lr_scale='linear', save_dir='weights/momentum/momentum-%s/' % str(momentum),
                          verbose=True)

#     # set the weight_decay here !
#     # lr doesnt matter as it will be over written by the callback
      optimizer = SGD(lr=0.0018, momentum=momentum, nesterov=True)
      model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#         # Fit the model on the batches generated by datagen.flow().
      model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks=[model_checkpoint]
)

# from plot we see, the model isnt impacted by the weight_decay very much at all
# so we can use any of them.

for momentum in MOMENTUMS:
    directory = 'weights/momentum/momentum-%s/' % str(momentum)

    losses, lrs = LRFinder.restore_schedule_from_dir(directory, 10, 5)
    plt.plot(lrs, losses, label='momentum=%0.2f' % momentum)

plt.title("Momentum")
plt.xlabel("Learning rate")
plt.ylabel("Validation Loss")
plt.legend()
plt.show()

NameError: ignored

In [0]:
# INITIAL WEIGHT DECAY FACTORS
# WEIGHT_DECAY_FACTORS = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7]

# FINEGRAINED WEIGHT DECAY FACTORS
WEIGHT_DECAY_FACTORS = [1e-7, 3e-7, 3e-6]

for weight_decay in WEIGHT_DECAY_FACTORS:
     lr_finder = LRFinder(n_train, 16, minimum_lr=0.0018, maximum_lr=1e-2,
                          validation_data=(X_test, Y_test),
                          validation_sample_rate=5,
                          lr_scale='linear', save_dir='weights/weight_decay/weight_decay-%s/' % str(weight_decay),
                          verbose=True)

#     # set the weight_decay here !
#     # lr doesnt matter as it will be over written by the callback
     optimizer = SGD(lr=0.0038, momentum=0.9, nesterov=True)
     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#
#         # Fit the model on the batches generated by datagen.flow().
     model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    callbacks=[model_checkpoint, ]
)

# from plot we see, the model isnt impacted by the weight_decay very much at all
# so we can use any of them.

for weight_decay in WEIGHT_DECAY_FACTORS:
    directory = 'weights/weight_decay/weight_decay-%s/' % str(weight_decay)

    losses, lrs = LRFinder.restore_schedule_from_dir(directory, 10, 5)
    plt.plot(lrs, losses, label='weight_decay=%0.7f' % weight_decay)

plt.title("Weight Decay")
plt.xlabel("Learning rate")
plt.ylabel("Validation Loss")
plt.legend()
plt.show()

Did not have time to try One Cycle LR.

<H1> EVALUATIONS </H1>

<H2>1ST SAVED MODEL - **BEST MODEL**</H2>

TOTAL ACCURACIES = 553.2909‬

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec.hdf5') #loss = 6.35742

In [16]:
evaluate_model(model)

32/32 [==============================] - 20s 620ms/step


{'age_output_acc': 44.3517,
 'bag_output_acc': 70.334,
 'emotion_output_acc': 70.0884,
 'footwear_output_acc': 67.2888,
 'gender_output_acc': 90.9627,
 'image_quality_output_acc': 60.6582,
 'pose_output_acc': 84.7741,
 'weight_output_acc': 64.833}

<h2>2ND SAVED MODEL - NOT TO BE CONSIDERED</H2>

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v1.hdf5') #loss = 7.34217

In [18]:
evaluate_model(model)

32/32 [==============================] - 11s 332ms/step


{'age_output_acc': 43.3694,
 'bag_output_acc': 63.998,
 'emotion_output_acc': 69.7446,
 'footwear_output_acc': 64.5874,
 'gender_output_acc': 81.336,
 'image_quality_output_acc': 58.6444,
 'pose_output_acc': 83.5462,
 'weight_output_acc': 62.0825}

<H2>3RD SAVED MODEL</H2>

TOTAL ACCURACIES = 539.6856

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v2_25Dec_v2.hdf5') # loss = 6.48211

In [20]:
evaluate_model(model)

32/32 [==============================] - 11s 333ms/step


{'age_output_acc': 43.1729,
 'bag_output_acc': 67.6326,
 'emotion_output_acc': 70.4322,
 'footwear_output_acc': 63.556,
 'gender_output_acc': 86.7878,
 'image_quality_output_acc': 57.5147,
 'pose_output_acc': 85.167,
 'weight_output_acc': 65.4224}

<H2>4TH SAVED MODEL - NOT TO BE CONSIDERED</H2>

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v3.hdf5') #loss = 7.25337

In [22]:
evaluate_model(model)

32/32 [==============================] - 11s 331ms/step


{'age_output_acc': 41.4538,
 'bag_output_acc': 60.0688,
 'emotion_output_acc': 70.5305,
 'footwear_output_acc': 52.2102,
 'gender_output_acc': 77.5049,
 'image_quality_output_acc': 57.3183,
 'pose_output_acc': 75.2456,
 'weight_output_acc': 64.7839}

<H2>5TH SAVED MODEL</H2>

TOTAL ACCURACIES = 550.7858

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_26Dec_v4.hdf5') # loss = 6.27273

In [24]:
evaluate_model(model)

32/32 [==============================] - 11s 330ms/step


{'age_output_acc': 44.4499,
 'bag_output_acc': 69.2534,
 'emotion_output_acc': 70.6778,
 'footwear_output_acc': 65.0295,
 'gender_output_acc': 90.4715,
 'image_quality_output_acc': 59.5776,
 'pose_output_acc': 85.609,
 'weight_output_acc': 65.7171}

<H2>6TH SAVED MODEL</H2>

TOTAL ACCURACIES = 545.9233‬

In [0]:
model.load_weights('/content/gdrive/My Drive/EIP_Assignment5_v3_27Dec_v5.hdf5') # loss = 6.45580

In [26]:
evaluate_model(model)

32/32 [==============================] - 11s 331ms/step


{'age_output_acc': 42.6817,
 'bag_output_acc': 69.7446,
 'emotion_output_acc': 70.5305,
 'footwear_output_acc': 64.9312,
 'gender_output_acc': 91.6503,
 'image_quality_output_acc': 56.5815,
 'pose_output_acc': 85.2161,
 'weight_output_acc': 64.5874}